In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
N_eigens = 14
datos_antigua_full = pd.read_csv("l_Unif.csv", delimiter=",", on_bad_lines='skip', usecols=tuple(range(26 + N_eigens)))
datos_nueva_full = pd.read_csv("a_Unif.csv", delimiter=",", on_bad_lines='skip', usecols=tuple(range(26 + N_eigens)))
datos_antigua = datos_antigua_full
datos_nueva = datos_nueva_full
N_datos = len(datos_nueva)
columnas_normalizar_a = list(datos_nueva.keys()[2:])
columnas_normalizar_l = list(datos_antigua.keys()[2:])
print(columnas_normalizar_a)
print(columnas_normalizar_l)

['bx', 'by', 'bz', 'C00', 'C01', 'C02', 'C03', 'C04', 'C05', 'C11', 'C12', 'C13', 'C14', 'C15', 'C22', 'C23', 'C24', 'C25', 'C33', 'C34', 'C35', 'C44', 'C45', 'C55', 'eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5', 'eig_6', 'eig_7', 'eig_8', 'eig_9', 'eig_10', 'eig_11', 'eig_12', 'eig_13']


In [3]:
def one_hottear(d_frame, cols_discretas):
    cols_nuevas = []
    for column in cols_discretas:
        posibles_valores = set(d_frame[column])
        for i in posibles_valores:
            d_frame[column + str(i)] = 0
            d_frame.loc[d_frame[column] == i, column + str(i)] = 1
            cols_nuevas.append(column + str(i))
        #fin for 
        del d_frame[column]
    #fin for 
    d_frame.loc[:, cols_nuevas + [col for col in d_frame.columns if col not in cols_nuevas]]
#fin función

In [4]:
one_hottear(datos_antigua, ["# Shape", "Cry_st"])
one_hottear(datos_nueva, ["# Shape", "Cry_st"])

In [5]:
Datos_train_l, Datos_test_l = train_test_split(datos_antigua, test_size = 0.4)
Datos_train_a, Datos_test_a = train_test_split(datos_nueva, test_size = 0.4)

In [6]:
def normalizar(dataSet, features, parametros = []):
    """
    Esta función resta la media y luego divide entre la desviación estandar de cada dato en cada columna. Si se le especifica la media y la desviación en los parámetros 
    Usa esa media y desviación dada, de lo contrario, calcula la media y la desviación de cada columna especificada en "features"
    @input: dataSet <pd.DataFrame>: Tabla de datos a la que se le va a dar normalización a la columans dadas
    @input: features <list>: lista de "features" o columnas que se van a normalizar
    @input: (opcional) parametros <dict>: En caso de que se quiera normalizar con una media y una desviación dada se especifican en un diccionario así: {'mileage':{'media': 2.9, 'desviacion': 7}}
    """
    if len(parametros) == 0:
        for feature in features:
            media = np.mean(dataSet[feature])
            desviacion = np.std(dataSet[feature])
            if desviacion != 0:
                dataSet[feature] = (dataSet[feature] - media)/desviacion
            #fin if 
        #fin for
    #fin if 
    else:
        for feature in features:
            media = parametros[feature]['media']
            desviacion = parametros[feature]['desviacion']
            if desviacion != 0:
                dataSet[feature] = (dataSet[feature] - media)/desviacion
            #fin if 
        #fin for
#fin función

In [7]:
Estadisticos_train_l= dict(map(lambda x: (x, {'media': np.mean(Datos_train_l[x]), 'desviacion': np.std(Datos_train_l[x])}), Datos_train_l.keys()))
Estadisticos_train_a= dict(map(lambda x: (x, {'media': np.mean(Datos_train_a[x]), 'desviacion': np.std(Datos_train_a[x])}), Datos_train_a.keys()))

In [8]:
normalizar(Datos_train_l, columnas_normalizar_l)
normalizar(Datos_train_a, columnas_normalizar_a)
normalizar(Datos_test_l, columnas_normalizar_l, Estadisticos_train_l)
normalizar(Datos_test_a, columnas_normalizar_a, Estadisticos_train_a)

In [9]:
"""
TODO: 
1) Hacer una lista con los features que serán nuestros X. Escoger cualquiera de los CXX y ese será nuestro 'y' (luego haremos un loop para cubrir todos los CXX)
2) Hacer una regresión lineal con cada uno de los CXX y reportar un MSE por cada uno de los CXX
"""
features_discretos = list(filter(lambda x: "Shape" in x or "CrySt" in x, Datos_train_a.keys()))
features_X_a = list(filter(lambda x: "eig" in x, Datos_train_a.keys()))
features_X_l = list(filter(lambda x: "omega" in x, Datos_train_l.keys()))
features_geo_l = ["Density", "Lx", "Ly", "Lz"]
features_geo_a = ["bx", "by", "bz"]
features_a = features_discretos + features_geo_a + features_X_a
features_l = features_discretos + features_geo_l + features_X_l
print(features_a)
print(features_l)

['# Shape0.0', '# Shape1.0', '# Shape2.0', 'bx', 'by', 'bz', 'eig_0', 'eig_1', 'eig_2', 'eig_3', 'eig_4', 'eig_5', 'eig_6', 'eig_7', 'eig_8', 'eig_9', 'eig_10', 'eig_11', 'eig_12', 'eig_13']
['# Shape0.0', '# Shape1.0', '# Shape2.0', 'Density', 'Lx', 'Ly', 'Lz', '(omega^2)_0', '(omega^2)_1', '(omega^2)_2', '(omega^2)_3', '(omega^2)_4', '(omega^2)_5', '(omega^2)_6', '(omega^2)_7', '(omega^2)_8', '(omega^2)_9', '(omega^2)_10', '(omega^2)_11', '(omega^2)_12']


In [10]:
X_train_a = Datos_train_a[features_a]
X_train_l = Datos_test_l[features_l]
X_test_a = Datos_test_a[features_a]
X_test_l = Datos_test_l[features_l]

In [11]:
targets = ["C00", "C11", "C22", "C33", "C44", "C55", "C01", "C02", "C12"]
MSE_a = dict()
MSE_l = dict()
W_a = dict()
W_l = dict()
for target in targets:
    y_train_a = Datos_train_a[target]
    y_train_l = Datos_test_l[target]
    y_test_a = Datos_test_a[target]
    y_test_l = Datos_test_l[target]
    modelo_a = LinearRegression()
    modelo_l = LinearRegression()
    modelo_a.fit(X_train_a, y_train_a)
    modelo_l.fit(X_train_l, y_train_l)
    w_a = [modelo_a.intercept_, *modelo_a.coef_]
    w_l = [modelo_l.intercept_, *modelo_l.coef_]
    y_gorro_test_a = X_test_a.dot(w_a[1:]) + w_a[0]
    y_gorro_test_l = X_test_l.dot(w_l[1:]) + w_l[0]
    W_a[target] = dict(map(lambda x: (features_a[x], w_a[1:][x]), range(len(features_a))))
    W_l[target] = dict(map(lambda x: (features_l[x], w_l[1:][x]), range(len(features_l))))
    W_a[target]["intercepto"] = w_a[0]
    W_l[target]["intercepto"] = w_l[0]
    W_a[target]["MSE"] = (1/(len(y_test_a)))*(y_test_a - y_gorro_test_a).dot(y_test_a - y_gorro_test_a)
    W_l[target]["MSE"] = (1/(len(y_test_l)))*(y_test_l - y_gorro_test_l).dot(y_test_l - y_gorro_test_l)
    media_a = Estadisticos_train_a[target]["media"]
    media_l = Estadisticos_train_l[target]["media"]
    desv_a = Estadisticos_train_a[target]["desviacion"]
    desv_l = Estadisticos_train_l[target]["desviacion"]
    nu_test_a = (media_a + desv_a*y_gorro_test_a)/(media_a + desv_a*y_test_a)
    nu_test_l = (media_l + desv_l*y_gorro_test_l)/(media_l + desv_l*y_test_l)
    W_a[target]["MSE_p"] = (1/(len(y_test_a)))*(1 - nu_test_a).dot(1 - nu_test_a)
    W_l[target]["MSE_p"] = (1/(len(y_test_l)))*(1 - nu_test_l).dot(1 - nu_test_l)

#fin for 
W_l = pd.DataFrame(W_l)
W_a = pd.DataFrame(W_a)

In [12]:
print("b y pesos para el problema adimensionalizado: ")
print(W_a)
nombre_archivo_a = "Regresion_lineal_a_" + str(N_datos) + ".csv"
W_a.to_csv(nombre_archivo_a)

b y pesos para el problema adimensionalizado: 
                 C00       C11       C22       C33       C44       C55  \
# Shape0.0  0.183499  0.175457  0.169061  0.296340  0.302618  0.297225   
# Shape1.0  0.048016  0.043074  0.030667  0.065168  0.066050  0.061369   
# Shape2.0 -0.231515 -0.218531 -0.199728 -0.361508 -0.368667 -0.358594   
bx          0.368131  0.359084  0.325705  0.464450  0.468370  0.461168   
by          0.353388  0.344463  0.335879  0.467283  0.466818  0.461997   
bz          0.375704  0.372552  0.341300  0.490392  0.494587  0.485341   
eig_0      -0.529921 -0.514428 -0.440516 -0.058358 -0.051182 -0.085085   
eig_1       0.261387  0.257094  0.193835 -0.399500 -0.401682 -0.318451   
eig_2       0.345761  0.318338  0.297734  0.158656  0.150373  0.164125   
eig_3      -0.124565 -0.070371 -0.114684 -0.258441 -0.246654 -0.153675   
eig_4      -0.997341 -0.995894 -0.996175 -0.386841 -0.405774 -0.606962   
eig_5       0.656086  0.610429  0.703939  0.924574  0.947440  0.8

In [13]:
print("b y pesos para el problema a la antigua")
print(W_l)
nombre_archivo_l = "Regresion_lineal_l_" + str(N_datos) + ".csv"
W_l.to_csv(nombre_archivo_l)

b y pesos para el problema a la antigua
                   C00       C11       C22       C33       C44       C55  \
# Shape0.0    0.039856  0.043575  0.026410  0.042101  0.040987  0.042360   
# Shape1.0   -0.008231 -0.013968 -0.016587 -0.002627  0.002536  0.000891   
# Shape2.0   -0.031625 -0.029606 -0.009824 -0.039474 -0.043523 -0.043251   
Density       0.039525  0.033617  0.036838  0.053046  0.051667  0.054533   
Lx            0.054425  0.046772  0.046842  0.054988  0.052193  0.054331   
Ly            0.037013  0.033678  0.037671  0.056572  0.056602  0.055211   
Lz            0.037875  0.037693  0.042474  0.064592  0.063600  0.063118   
(omega^2)_0  -0.139496 -0.160866 -0.075844  0.095861  0.099984  0.089823   
(omega^2)_1   0.171547  0.198446  0.150901 -0.150493 -0.141881 -0.073009   
(omega^2)_2   0.082872  0.086692  0.050775 -0.033389 -0.027186 -0.032468   
(omega^2)_3  -0.009212  0.004614 -0.012188  0.015368  0.008382 -0.044318   
(omega^2)_4  -0.144774 -0.178855 -0.294137  0.16

In [15]:
print(W_a.T["MSE"])

C00    0.621361
C11    0.629739
C22    0.658600
C33    0.346101
C44    0.348661
C55    0.378168
C01    0.904147
C02    0.905896
C12    0.905303
Name: MSE, dtype: float64
